In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import h5py
import numpy as np
from matplotlib import pyplot as plt

from utils.training_state import TrainingState
from models.data import data_partition

# Read data

In [118]:
with h5py.File("local_data/re100/ux.h5",'r') as hf:
    ux = np.array(hf.get("ux"))
with h5py.File("local_data/re100/uy.h5",'r') as hf:
    uy = np.array(hf.get("uy"))
with h5py.File("local_data/re100/pp.h5",'r') as hf:
    pp = np.array(hf.get("pp"))

In [22]:
import jax
key1 = jax.random.PRNGKey(10)
[key3] = jax.random.split(key2,1)
key3

DeviceArray([ 913281243, 1281629655], dtype=uint32)

In [23]:
import jax.numpy as jnp

a = jnp.array(jnp.arange(10).reshape((5,2)))
b = jax.random.permutation(key1,a,axis=0,independent=False)
print(a)
print(b)
c = jax.random.permutation(key3,a,axis=0,independent=False)
print(c)

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
[[4 5]
 [0 1]
 [8 9]
 [6 7]
 [2 3]]
[[2 3]
 [6 7]
 [8 9]
 [4 5]
 [0 1]]
